In [1]:
import asyncio
import aiohttp
import pandas
import nest_asyncio
nest_asyncio.apply()

In [2]:
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
}

import time
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
SURL = "http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=page_change&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=0&f=S&l=50&TERM1=search_change&FIELD1=&co1=AND&TERM2=&FIELD2=&d=PTXT"
key_words = ["wall plant","Plant hanger","plant holder"]
# key_words = ["wall plant"]

In [4]:
key_urls = {}
for key_word in key_words:
    key_words_url = SURL
    key_words_url = key_words_url.replace("search_change",str(key_word))
    url_1 = key_words_url
    url_1 = url_1.replace("page_change","1")
    key_urls[key_word]=url_1

In [5]:
url_lst_failed = []
url_lst_successed = []
pages_key_urls = {}

async def get_page(session,key):
    async with session.get(key_urls[key], timeout=300) as resp:
        if resp.status != 200:
            url_lst_failed.append(key_urls[key])
        else:
            url_lst_successed.append(key_urls[key])
        return await resp.text(),key

In [6]:
async def parser(html,key):
    page_a = []
    page_img = []
    page_href = []
    soup = BeautifulSoup(html, 'html.parser')
    total_len = soup.find("body").find_all("i")[1].find_all("strong")[2].get_text()
    page_num = int(float(total_len)/50.5)+2
    print(key+"一共"+total_len+"个论文，共"+str(page_num-1)+"页")
    if page_num >= 2:
        for i in range(2,page_num):
            key_words_url = SURL
            key_words_url = key_words_url.replace("search_change",str(key))
            url_i = key_words_url
            url_i = url_i.replace("page_change",str(i))
            if key not in pages_key_urls:
                pages_key_urls[key] = [url_i]
            else:
                pages_key_urls[key].append(url_i)
    soup_tb = soup.find_all("table")
    for tb in soup_tb:
        for tr in tb.find_all("tr"):
            valign_top = list(tr.find_all("td",attrs={"valign":"top"}))
            if len(valign_top)>=2:
                num = valign_top[1].get_text().replace(",","")
                img = "https://pdfpiw.uspto.gov/.piw?Docid="+str(num)
                top = valign_top[2]
                href = "http://patft.uspto.gov/"+str(top.a.get("href"))
                a = top.get_text()
                a = a.replace("\n"," ")
                page_a.append(a)
                page_img.append(img)
                page_href.append(href)
    page_dict = {"标题":page_a,"专利链接":page_href,"图片链接":page_img}
    page_df = pd.DataFrame(page_dict)
    page_df.to_csv(key+".csv",index=None)
    print("导出"+key+"第一页")


In [7]:
async def get_page_s(session,key,url):
    async with session.get(url, timeout=60) as resp:
        if resp.status != 200:
            url_lst_failed.append(url)
        else:
            url_lst_successed.append(url)
        return await resp.text(),key

In [8]:
async def parser_s(html,key):
    page_a = []
    page_img = []
    page_href = []
    soup = BeautifulSoup(html, 'html.parser')
    soup_tb = soup.find_all("table")
    for tb in soup_tb:
        for tr in tb.find_all("tr"):
            valign_top = list(tr.find_all("td",attrs={"valign":"top"}))
            if len(valign_top)>=2:
                num = valign_top[1].get_text().replace(",","")
                img = "https://pdfpiw.uspto.gov/.piw?Docid="+str(num)
                top = valign_top[2]
                href = "http://patft.uspto.gov/"+str(top.a.get("href"))
                a = top.get_text()
                a = a.replace("\n"," ")
                page_a.append(a)
                page_img.append(img)
                page_href.append(href)
    page_dict = {"标题":page_a,"专利链接":page_href,"图片链接":page_img}
    page_df = pd.DataFrame(page_dict)
    page_df.to_csv(key+".csv",index=None,mode='a',header=False)
    print(key+" +1")

In [9]:
async def download(key):
    async with aiohttp.ClientSession() as session:
        html,key = await get_page(session,key)
        await parser(html,key)

In [10]:
async def download_s(key,url):
    async with aiohttp.ClientSession() as session:
        html,key = await get_page_s(session,key,url)
        await parser_s(html,key)

In [11]:
start = time.time()

tasks = [asyncio.ensure_future(download(key)) for key in key_urls]
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(tasks))

tasks_s = []
for key in pages_key_urls:
    for url in pages_key_urls[key]:
        tasks_s.append(asyncio.ensure_future(download_s(key,url)))
loop_s = asyncio.get_event_loop()
loop_s.run_until_complete(asyncio.wait(tasks_s))
print("#"*40)
end = time.time()
print('总共耗时{}秒'.format(end-start))

Plant hanger一共127个论文，共3页
导出Plant hanger第一页
plant holder一共249个论文，共5页
导出plant holder第一页
wall plant一共441个论文，共9页
导出wall plant第一页
plant holder +1
wall plant +1
Plant hanger +1
Plant hanger +1
wall plant +1
plant holder +1
plant holder +1
wall plant +1
plant holder +1
wall plant +1
wall plant +1
wall plant +1
wall plant +1
wall plant +1
########################################
总共耗时36.6012704372406秒
